In [1]:
import pandas as pd
import numpy as np
import rake
import re
import math
import itertools, nltk, string

# Data download, preliminary clean, explore

In [3]:
PRF = pd.read_csv('/Users/yingjiang/Dropbox/Learnings/Stats_data/Projects/IMRE_work/Manpower/PRF.csv')
staff = pd.read_csv('/Users/yingjiang/Dropbox/Learnings/Stats_data/Projects/IMRE_work/Manpower/Staff.csv')

## PRF data

In [4]:
PRF.head(10)

Title of Paper  \
0  Recent Progress in Chemical Vapor Deposition G...   
1  A Polydopamine Coating Ultralight Graphene Mat...   
2  MOLECULAR DESIGN OF BIOINSPIRED NANOSTRUCTURES...   
3  Biodegradable Thermogelling Polymers for Biome...   
4  Dual-responsive Reversible Photo/Thermogelling...   
5  Efficient and Tunable Plasmon Excitation in Gr...   
6  Growth and characterization of highly tensile ...   
7  Investigation of surface band bending of Ga-fa...   
8  Rational Design of Biomolecular Templates for ...   
9  Bovine Serum Albulmin Protein-Templated Silver...   

                                       Journal Title  \
0  Progress in Crystal Growth and Characterizatio...   
1                                 Renewable Energy     
2   Journal of Molecular and Engineering Materials     
3                                     MRS Bulletin     
4  Journal of Polymer Science, Part A: Polymer Ch...   
5      Journal of the Optical Society of America B     
6                       Journal of Applied Physics     
7  ECS Journal of Solid State Science and Technol...   
8                    Advanced Healthcare Materials     
9                    Advanced Healthcare Materials     

  Publication Release Number                     Dept/Prog   \
0               MPC/16-128                           -MPC     
1                AS/16-056    Advanced Energy Storage Lab     
2                BB/16-185       -Biomedical & Biomimetic     
3                CT/16-229      -Consumer Care Technology     
4                CT/16-227      -Consumer Care Technology     
5               MET/16-146                           -MET     
6                AC/14-835                             AC     
7                AC/15-648                             AC     
8                BB/16-161       -Biomedical & Biomimetic     
9                BB/16-160       -Biomedical & Biomimetic     

                               Significance of Paper Publication Date  \
0  Reviews the latest progress in CVD growth of T...           Jul-16   
1  The soft polydopmine (PDA) layer with polar fu...           Aug-16   
2  In this review, we present the recent developm...           Aug-16   
3  Thermogelling polymers belong to a class of st...           Aug-16   
4  This work describes the synthesis of a novel p...           Jun-16   
5  This work reported a strong plasmonic coupling...           Sep-16   
6  Ge0.94Sn0.06 films with high tensile strain we...           Mar-16   
7  The surface band bending of Ga-face n-type GaN...           Aug-16   
8  This progress report reviews the state-of-the-...           Jul-16   
9  Bovine serum albumin-templated silver nanoclus...           Jul-16   

             First Author    First Author Organisation  \
0       WONG Swee Liang                         IMRE     
1              Zhou Lan             Fudan University     
2   stu-victorxu (IMRE)                         IMRE     
3  LIOW Sing Shy (IMRE)                         IMRE     
4  DOU Qing Qing (IMRE)                         IMRE     
5                Yao Lu    Soochow University, China     
6              Wei Wang                          NUS     
7            T. L. Duan                          NTU     
8               YU Yong                         IMRE     
9               YU Yong                         IMRE     

                                     Other Author(s)  \
0                        LIU Hong Fei, CHI Dongzhi     
1  LIU Zhao Lin (IMRE), ZONG Yun (IMRE), Yu Aishui     
2  ZHENG Xinting, Beverly MOK Yin Leng, stu-salwa...   
3     Anis ABDUL KARIM (IMRE), LOH Xian Jun (IMRE)     
4  LIOW Sing Shy (IMRE), LOH Xian Jun (IMRE), Wen...   
5  Steve WU Qing Yang, ZHAO Meng, TENG Jing Hua, ...   
6  PAN Ji Sheng (IMRE), ZHANG Zheng (IMRE), Wan K...   
7                   PAN Ji Sheng (IMRE), D. S. Ang     
8  Beverly MOK Yin Leng, LOH Xian Jun, TAN Yen Nee     
9  TAN Yen Nee, CHELLAPPAN Vijila, Geng Junlong, ...   

                        Other Author(s) organisati

In [4]:
PRF.shape

(3313, 15)

In [55]:
print sum(PRF.First_Author.isnull())
print sum(PRF.Other_Authors.isnull())
print sum(PRF.Corresponding_Authors.isnull())

2
23
1325


In [5]:
PRF.columns

Index([u'Title of Paper', u'Journal Title', u'Publication Release Number',
       u'Dept/Prog ', u'Significance of Paper', u'Publication Date',
       u'First Author', u'First Author Organisation', u'Other Author(s)',
       u'Other Author(s) organisation', u'Corresponding Author(s)',
       u'Corresponding Author(s) Organisation(s)', u'Project Finance Code',
       u'Project Title', u'Web ID'],
      dtype='object')

Remove spaces in columns:

In [4]:
def cleanColNames(vector = PRF.columns):
    newCol = []
    for i in vector:
        i = i.replace(' ', '_')
        i = i.replace('(', '')
        i = i.replace(')', '')
        i = i.replace('/', '_or_')
        newCol.append(i)
    return newCol

In [5]:
PRF.columns = cleanColNames(PRF.columns)
print PRF.columns

Index([u'Title_of_Paper', u'Journal_Title', u'Publication_Release_Number',
       u'Dept_or_Prog_', u'Significance_of_Paper', u'Publication_Date',
       u'First_Author', u'First_Author_Organisation', u'Other_Authors',
       u'Other_Authors_organisation', u'Corresponding_Authors',
       u'Corresponding_Authors_Organisations', u'Project_Finance_Code',
       u'Project_Title', u'Web_ID'],
      dtype='object')


In [479]:
'''indRecent = []
for ind, val in PRF.Publication_Date.iteritems():
    print ind, val, '\n'
    if str(val).lower() == 'nan':
        if ('16' in val) | ('15' in val):
            indRecent.append(ind)'''

0 Jul-16 

1 Aug-16 

2 Aug-16 

3 Aug-16 

4 Jun-16 

5 Sep-16 

6 Mar-16 

7 Aug-16 

8 Jul-16 

9 Jul-16 

10 Aug-16 

11 Aug-16 

12 Apr-16 

13 Apr-16 

14 Jun-16 

15 Aug-16 

16 Jul-16 

17 May-16 

18 Dec-15 

19 Nov-15 

20 Jan-16 

21 Jun-16 

22 Jul-16 

23 Jul-16 

24 Jul-16 

25 Jul-16 

26 Jul-16 

27 Jul-16 

28 Jul-16 

29 Jul-16 

30 May-16 

31 Apr-16 

32 Jul-16 

33 Jan-16 

34 Jul-16 

35 Jul-16 

36 Apr-16 

37 Jun-16 

38 Apr-16 

39 Feb-16 

40 Aug-15 

41 Jul-16 

42 Apr-16 

43 Jul-16 

44 Jul-16 

45 Jul-16 

46 Jul-16 

47 Apr-16 

48 May-16 

49 Jun-16 

50 Apr-16 

51 Jul-16 

52 Jul-16 

53 Jul-16 

54 Jul-16 

55 Jul-16 

56 Apr-16 

57 Jan-16 

58 Apr-16 

59 Feb-16 

60 Jun-16 

61 Jun-16 

62 Jun-16 

63 Jun-16 

64 Jun-16 

65 May-16 

66 Mar-16 

67 Jun-16 

68 Nov-15 

69 May-16 

70 May-16 

71 May-16 

72 May-16 

73 May-16 

74 May-16 

75 May-16 

76 May-16 

77 May-16 

78 Apr-16 

79 Mar-16 

80 May-16 

81 May-16 

82 Feb-16 

83 May-16 

84

TypeError: argument of type 'float' is not iterable

In [21]:
len(PRF.First_Author.unique())

2128

In [104]:
PRF.First_Author.loc[PRF.First_Author_Organisation.isnull()]

2312                     NaN
3145         Benzhong Wang  
3152               H F Liu  
3153         Benzhong Wang  
3154         Benzhong Wang  
3155               S T Tan  
3158                     NaN
3161             3. W.Zhao  
3163               H F Liu  
3164              L.J. Jin  
3165             S.L. Liew  
3166              E.J. Tan  
3168      Q.F. Daphne Yiew  
3169             S.L. Liew  
3170                Rui Li  
3174              Y Z Zhao  
3176               Chen XT  
3191                J W Yu  
3192               C Y Liu  
3202                W Tian  
3213           Bratland KA  
3222                Jin LJ  
3223    Md. Anisur Rahmana  
3224             Rahman MA  
3225             H. P. Sun  
3226            Loomans ME  
3228               Law CMT  
3229             Anisur MR  
3230                 Ee YC  
3236              Q.F. Yan  
3237             X.C. Wang  
3238             X.C. Wang  
3243               Q F Fan  
3248               W C GOH  
3250          

In [467]:
PRF.Publication_Date

0       Jul-16
1       Aug-16
2       Aug-16
3       Aug-16
4       Jun-16
5       Sep-16
6       Mar-16
7       Aug-16
8       Jul-16
9       Jul-16
10      Aug-16
11      Aug-16
12      Apr-16
13      Apr-16
14      Jun-16
15      Aug-16
16      Jul-16
17      May-16
18      Dec-15
19      Nov-15
20      Jan-16
21      Jun-16
22      Jul-16
23      Jul-16
24      Jul-16
25      Jul-16
26      Jul-16
27      Jul-16
28      Jul-16
29      Jul-16
         ...  
3283      2005
3284      2005
3285      2005
3286      2005
3287      2005
3288      2005
3289      2004
3290      2004
3291      2005
3292      2005
3293      2005
3294      2005
3295      2005
3296      2005
3297      2005
3298      2005
3299      2005
3300      2005
3301      2005
3302      2005
3303      2005
3304      2005
3305      2005
3306      2005
3307      2005
3308      2005
3309      2005
3310      2005
3311      2005
3312      2005
Name: Publication_Date, dtype: object

## Staff data

In [89]:
staff.head()

Staff Name        Title   Dpt Functions  \
0               Afriyanti SUMBOJA  Scientist I  -CER    #NAME?   
1          Agata Maria BRZOZOWSKA  Scientist I  -MOL    #NAME?   
2             Ajay Kumar KUSHWAHA  Scientist I  -CER    #NAME?   
3                          AN Tao  Scientist I  -CER    #NAME?   
4  Anas Ibrahim SaedAldin AbuTaha  Scientist I  -CER    #NAME?   

   Function Involvement  Project Involvement  Total involvement  
0                   0.0                  1.0                1.0  
1                   0.0                  0.5                0.5  
2                   0.0                  0.0                0.0  
3                   0.0                  1.0                1.0  
4                   0.0                  1.0                1.0

In [6]:
staff.columns = cleanColNames(vector = staff.columns)
print staff.columns

Index([u'Staff_Name', u'Title', u'Dpt', u'Functions', u'Function_Involvement',
       u'Project_Involvement', u'Total_involvement'],
      dtype='object')


In [412]:
staff.iloc[290:, :]

Staff_Name               Title   Dpt Functions  Function_Involvement  \
290   ZHOU Hui         Scientist I  -POL    #NAME?                   0.0   
291  ZHU Qiang         Scientist I  -POL    #NAME?                   0.0   
292   ZONG Yun  Senior Scientist I  -MOL    #NAME?                   0.0   
293        NaN                 NaN   NaN       NaN                   NaN   
294        NaN                 NaN   NaN       NaN                   NaN   
295        NaN                 NaN   NaN       NaN                   NaN   
296  Total FTE                 293   NaN       NaN                   0.0   
297        NaN                   1   NaN       NaN                   0.0   

     Project_Involvement  Total_involvement  
290             1.000000           1.000000  
291             1.000000           1.000000  
292             1.000000           1.000000  
293                  NaN                NaN  
294                  NaN                NaN  
295                  NaN                NaN  
296           193.230000         193.230000  
297             0.659488           0.659488

In [7]:
staff.drop(range(293,298), axis = 0, inplace = True)

In [400]:
# Some first authors have affiliation secondary to IMRE.
for ind, i in enumerate(PRF_IMRE.First_Author_Organisation):
    if (',' in i) & ('IMRE' in i):
        print ind
        print firstIMRE[ind]
        print i
        print corrIMRE[ind]
        print PRF_IMRE.Corresponding_Authors_Organisations.values[ind]

61
['LAY Chee leng']
NTU, CBC; IMRE  
['PHANG In Yee', 'LING Xing Yi']
IMRE, NTU, CBC; IMRE 
166
['Guijian Guan']
IHPC, and attached to IMRE  
['HAN Ming Yong', 'YongWei Zhang']
IMRE, IHPC 
195
['Xing Zheng']
NUS, IMRE student  
['HONG Liang']
IMRE 
220
['Chee Leng Lay']
NTU, CBC; IMRE  
['Xing Yi Ling']
NTU, CBC 
247
['Huajun Liu']
IMRE and NUS (out station, in US for two years)  
['YAO Kui', 'John Wang']
IMRE, NUS 
375
['Steven Lukman']
former IMRE staff, now studying in UK  
['SU Xiao Di']
IMRE 
380
['Ng Yeap Hung']
NUS, IMRE student  
['HONG Liang']
IMRE 
446
['K Shubhakar']
SUTD, NTU, IMRE  
['K Shubhakar']
SUTD, NTU, IMRE 
611
['Hiang Kwee LEE']
CBC, NTU; IMRE, A*Star  
['Ling Xing Yi']
CBC, NTU 
615
['Guan Guijian']
IHPC, attached to IMRE  
['HAN Ming Yong', 'YongWei Zhang']
IMRE, IHPC 
628
['Naveen']
NUS, IMRE PhD student  
['CHELLAPPAN Vijila']
IMRE 
817
['Naveen Kumar Elumalai']
NUS, IMRE PhD student  
['CHELLAPPAN Vijila']
IMRE 
895
['Lining He']
NTU, IMRE  
['JIANG Changyun

# Clean author data

## Get inidividual authors

In [8]:
# Function to break text into a list of authors

'''
# Debugging version with print statements
nonFirst = []
for ind, i in enumerate(PRF.Other_Authors):
    print ind
    print 'All non-first authors: ', i
    if str(i).lower() == 'nan':
        nonFirst.append('No other authors')
    else:
        tmp = []
        for author in i.split(','):
            print author
            if '(' in author:
                tmp.append(author.split('(')[0].strip())
            else:
                tmp.append(author.strip())
            print tmp
        nonFirst.append(tmp)
        print nonFirst
        print '\n'

print nonFirst
'''
'''
nonFirst = []
for ind, i in enumerate(PRF.Other_Authors):
    if str(i).lower() == 'nan':
        nonFirst.append('No other authors')
    else:
        tmp = []
        for author in i.split(','):
            if '(' in author:
                tmp.append(author.split('(')[0].strip())
            else:
                tmp.append(author.strip())
        nonFirst.append(tmp)
'''

def makeIndvAuth(vector = PRF.Other_Authors):
    authors = []
    for ind, i in enumerate(vector):
        if str(i).lower() == 'nan':
            authors.append('No other authors')
        else:
            tmp = []
            for author in i.split(','):
                if '(' in author:
                    tmp.append(author.split('(')[0].strip())
                else:
                    tmp.append(author.strip())
            authors.append(tmp)
    return authors

### Get inidividual authors in "First_Authors"

In [9]:
first = makeIndvAuth(PRF.First_Author)

In [43]:
## Removed special characters for 2 names.
for i in first:
    if 'Milo' in i[0]:
        i[0] = 'Milos Petrovic'
#         print first.index(i)
#         print 'String is now changed to', i[0]
    if 'Mechthild' in i[0]:
        i[0] = 'Mechthild Lubke'
#         print first.index(i)        
#         print 'String is now changed to', i[0]

In [10]:
print first

[['WONG Swee Liang'], ['Zhou Lan'], ['stu-victorxu'], ['LIOW Sing Shy'], ['DOU Qing Qing'], ['Yao Lu'], ['Wei Wang'], ['T. L. Duan'], ['YU Yong'], ['YU Yong'], ['Liu Junyi'], ['David'], ['Zhang Yu'], ['Ang Huixiang'], ['Fan Zhanxi'], ['Stephen John Fox'], ['Ghislaine Barouti'], ['Ako Rajour Tanyi1'], ['Milos Petrovic'], ['Milo\x81\xc1 Petrovic'], ['Milos Pertrovic'], ['P. D. Tran'], ['R. P. Antony'], ['Goutam Kumar DALAPATI'], ['zhang chong jing'], ['Yuan you yong'], ['YANG Zhe'], ['GUO Shifeng'], ['Jae Yoon Hwang'], ['Roozbeh Siavash Moakhar'], ['Gan Lu'], ['LI Haidong'], ['HE Jiating'], ['LIU Rong Rong'], ['WAi Hoe Tham'], ['WANG Shengqin'], ['Jun Zhou'], ['LIU Hongwei'], ['Yangjie Huang'], ['Miao Qi'], ['Antonina M. Monaco'], ['David PARAMELLE'], ['YANG Ming'], ['Neerajn Dwivedi'], ['S. Arulkumaran'], ['Avishek Das'], ['Zhan Yi'], ['Neo Wei Teng'], ['YE Qun'], ['Ri Rong'], ['Bo He'], ['Zhou Lan'], ['LEE Chee Wei'], ['Yang He'], ['Li Chengguo'], ['Romen Herrera Padilla'], ['wei tang'

### Get inidividual authors in "Corresponding_Authors"

In [11]:
corr = makeIndvAuth(vector = PRF.Corresponding_Authors)

In [245]:
print corr

[['CHI Dongzhi'], ['LIU Zhao Lin', 'ZONG Yun', 'Yu Aishui'], ['TAN Yen Nee'], ['LOH Xian Jun'], ['LOH Xian Jun'], ['Qiaoliang Bao'], ['Yee-Chia Yeo'], ['PAN Ji Sheng', 'D. S. Ang'], ['TAN Yen Nee'], ['TAN Yen Nee'], ['REN Yi', 'CHIAM Sing Yang', 'Li Fong Yau Sam'], ['LOH Xian Jun'], ['Yan Qingyu'], ['Yan Qingyu'], ['Prof. Zhang Hua'], ['LOH Xian Jun', 'Chandra Shekhar Verma', 'Navin Kumar Verma', 'Rajamani Lakshminarayanan'], ['LOH Xian Jun', 'Sophie M. Guillaume'], ['Piyasiri Ekanayake'], ['CHELLAPPAN Vijila', 'Prof. Seeram Ramakrisha'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'], ['P. D. Tran', 'V. Artero'], ['S. C. J. Loo', 'J. Barber', 'L. H. Wong'], ['Goutam Kumar DALAPATI'], ['LIU Bin'], ['LIU Bin'], ['LING Xing Yi'], ['Dominik Janczewski', 'G. Julius Vancso'], ['LI Zibiao', 'LOH Xian Jun'], ['Gregory GOH Kia Liang'], ['Dominik Janczewski', 'Christian A. Nijhuis'], ['LI Xu', 'LIU Hong'], ['LI Xu'], ['Nikodem TOMCZAK'], ['TRI

### Get inidividual authors in "Other_Authors"

In [13]:
nonFirst = makeIndvAuth(PRF.Other_Authors)

#### Take out corresponding authors in "Other_Authors"

In [17]:
for otherAuth, corrAuth in zip(nonFirst, corr):
    print nonFirst.index(otherAuth)
    
    for author in otherAuth:
        if author in corrAuth:
            print author
            otherAuth.remove(author)

0
CHI Dongzhi
1
LIU Zhao Lin
Yu Aishui
2
TAN Yen Nee
3
LOH Xian Jun
4
LOH Xian Jun
5
Qiaoliang Bao
6
Yee-Chia Yeo
7
PAN Ji Sheng
8
TAN Yen Nee
9
TAN Yen Nee
10
REN Yi
CHIAM Sing Yang
11
LOH Xian Jun
12
Yan Qingyu
13
Yan Qingyu
14
Prof. Zhang Hua
15
LOH Xian Jun
Chandra Shekhar Verma
Rajamani Lakshminarayanan
16
LOH Xian Jun
Sophie M. Guillaume
17
Piyasiri Ekanayake
18
CHELLAPPAN Vijila
19
CHELLAPPAN Vijila
Prof Seeram Ramakrishna
20
CHELLAPPAN Vijila
21
V. Artero
22
S. C. J. Loo
L. H. Wong
23
24
LIU Bin
25
LIU Bin
26
LING Xing Yi
27
Dominik Janczewski
G. Julius Vancso
28
LI Zibiao
29
Gregory GOH Kia Liang
30
Dominik Janczewski
31
LI Xu
LIU Hong
32
LI Xu
33
Nikodem TOMCZAK
34
TRIPATHY Sudhiranjan
Diing Shenp Ang
35
Yingxi Zhu
36
Yuan Ping Feng
37
Lu Junpeng
SOW Chorng Haur
38
Kuo-Wei Huang
39
David J. Young
ZHANG Zhongxing
40
Michele Giugliano
41
Jatin Nitin KUMAR
LIU Ye
42
WANG Shi Jie
43
Charanjit S. Bhatia
44
45
Goutam Kumar DALAPATI
46
Jim Yang Lee
47
XU Jian Wei
48
XU Jian Wei
49
X

AttributeError: 'str' object has no attribute 'remove'

#### Removed special characters for 1 names.

In [18]:
for i in nonFirst:
    for j in i:
        if 'Costa' in j:
            print nonFirst.index(i), i.index(j)
            j = 'Vijay Richard De Costa'

            print 'String is now changed to', j


6 4
String is now changed to Vijay Richard De Costa


In [19]:
print nonFirst

[['LIU Hong Fei'], ['ZONG Yun'], ['ZHENG Xinting', 'Beverly MOK Yin Leng', 'stu-salwaibrahim', 'YU Yong'], ['Anis ABDUL KARIM'], ['LIOW Sing Shy', 'Weng Wengui'], ['Steve WU Qing Yang', 'ZHAO Meng', 'TENG Jing Hua', 'Jingchao Song', 'Jian Yuan', 'Lei Zhang', 'Wenzhi Yu', 'Kian Ping Loh', 'Chao Zhang'], ['PAN Ji Sheng', 'ZHANG Zheng', 'Wan Khai Loke', 'Tingting Yin', 'Vijay Richard D\x89\xdb\xaa Costa', 'Qian Zhou', 'Yuan Dong', 'Gengchiau Liang', 'Zexiang Shen', 'Soon Fatt Yoon', 'Eng Soon Tok'], ['D. S. Ang'], ['Beverly MOK Yin Leng', 'LOH Xian Jun'], ['CHELLAPPAN Vijila', 'Geng Junlong', 'Edward Ong'], ['Nancy WONG Lai Mun', 'Li Fong Yau Sam', 'Gurudayal', 'Wong Lydia Helena'], ['LI Zibiao', 'YE Enyi'], ['LI Bing', 'ZONG Yun', 'Wang Huanwen', 'Luo Zhongzhen', 'Tan Huiteng', 'Sun Shengnan', 'Li Zhong', 'Xu Zhichuan', 'Khor Khiam Aik'], ['LI Bing', 'Wang Huanwen', 'Wang xuefeng', 'ZONG Yun'], ['LI Bing', 'ZONG Yun', 'Luo zhimin', 'Chen Ye', 'Wang Jie'], ['Mobashar Hussain Urf Turabe Fa

At this point, we have three author lists:
1. First authors: 'first', list of single-element lists
2. Corresponding authors 'corr', list of multiple-element lists
3. Other authors 'nonFirst', list of multiple-element lists

## Normalize first author names

Conclusions written ahead:

1. 2312 and 3158 have unknown authors for both first and corr. REMOVE these rows later.

2. 2034, 2051 and 2164 have the first and corr joined together (an intelligent guess would be the first author is the First and last is the corr). ANALYZE the organization for these rows later:
2034 - First is IMRE, corr is not
2051 - First is IMRE, corr is not
2164 - Both are IMRE

3. After removing all non-alphabets, the authors take the form:
- AB Surname
- A B Surname
- Surname AB
- Surname A B
- A Surname B

### Inspect null rows


In [136]:
# Fill null entries with a string for easier processing.
PRF.loc[:, ['First_Author', 'First_Author_Organisation', 'Corresponding_Authors', 'Corresponding_Authors_Organisations']] = PRF.loc[:, ['First_Author', 'First_Author_Organisation', 'Corresponding_Authors', 'Corresponding_Authors_Organisations']].fillna('UNKNOWN')

In [128]:
'''for ind, i in enumerate(PRF.First_Author):
    if str(i).lower() == 'nan':
        print ind
'''

for ind, i in enumerate(PRF.First_Author):
    if i == 'UNKNOWN':
        print ind


2312
3158


### Inspect number of authors

In [12]:
for ind, i in enumerate(first):
    if len(i) > 1:
        print ind, i

273 ['Yao', 'Xiaojing']
310 ['He', 'Xue']
386 ['Ma', 'H. J. Harsan']
634 ['C. G. Hou', '']
664 ['Tan Tsai Yuan', 'Tristan']
864 ['Toh', 'Cher Ling']
1353 ['Tan', 'Lihan']
1390 ['Kyaw', 'AKK']
1471 ['Wu', 'Shixin']
1473 ['Yang', 'W. F']
1474 ['Pan', 'Jun']
1510 ['Jia', 'Pengtao']
1560 ['Teo Kim Kiat', 'Benjamin']
1596 ['Phyllis S. Y. Lim', '']
1602 ['N. Raghavan', '']
1612 ['Bao', 'Qiaoliang']
1613 ['Zhang', 'Wei']
1741 ['Yang', 'MF']
1746 ['Tan Cheng Cheh', 'Dennis']
1767 ['A. Huang', '']
1912 ['Tong', 'Yuejin']
1982 ['HONG', 'Young-Rae']
2034 ['Xiaodong Zhou', 'Nan Zhang', 'Christina Tan', 'Wolfgang Knoll']
2051 ['William R. Birch', 'Heow Pueh Lee']
2061 ['Cher Ling Toh', '']
2122 ['Tey', 'JN']
2164 ['Yang Xiao', 'Xuehong Lu', 'Xinhai Zhang and Chaobin He']
2222 ['K. P. PRAMODA', '']
2245 ['Khine Y. Mya', '']
2246 ['Khine Y. Mya', '']
2300 ['', 'Xu Li*', '']
2301 ['D Janczewski', '']
2312 No other authors
2530 ['P Sonar', 'S P Singh']
2717 ['JH Yao', 'KY Mya']
2733 ['G. K. Dalapati', 

In [175]:
for ind, i in enumerate(PRF.Corresponding_Authors):
#     if str(i).lower() == 'nan':
    if i == 'UNKNOWN':
        print ind
        print nonFirst[ind]
        print PRF.Other_Authors_organisation.values[ind]

1918
['Tjiu Weng Weei', 'Liu Tianxi', 'Lui Wai Yi', 'Phang In Yee', 'Zhang Wei-De']
ASTAR, Inst Mat Res & Engn, Singapore 117602, Singapore; Fudan Univ, Key Lab Mol Engn Polymers, Minist Educ, Dept Macromol Sci, Shanghai 200433, Peoples R  
1927
['Gao HT', 'Ye EY', 'Low M', 'Lim SH', 'Lieu XH', 'Tripathy S', 'Tremel W', 'Han MY']
NUS/Univeristy of Mainz 
1930
['Robert H. Pelton Kam C. Tam']
Department of Chemical Engineering, McMaster University, Canada 
1931
['Beng H Tan Y Bai J Ma XH Lu']
NTU 
1932
['Hazrat Hussain Chaobin He']
IMRE 
1933
['Hoi Lam Tam', 'Kian Soon Yong', 'Zhi-Kuan Chen', 'Furong Zhu']
IMRE and HKBU 
1935
['Happy Tan', 'Miao Wang', 'Chang-Tong Yang', 'Shilpa Pant', 'Kishore Kumar Bhakoo', 'Siew Yee Wong', 'Zhi-Kuan Chen', 'Xu Li*', 'and John Wang']
Johns Hopkins International Medical Center; Laboratory of Molecular Imaging, Singapore Bioimaging Consortium ; National University of Singapore; IMRE 
1936
['SW Tay', 'ZL Liu', 'L Hong']
IMRE 
1966
['Lim Siew Lay', 'Tan Hu

In [130]:
## Clean up for rows 2034, 2051, 2164
## These rows still consist of more than one first author upon first splitting.
print first[2034], corr[2034]
print first[2051], corr[2051]
print first[2164], corr[2164]
print first[2312], corr[2312]
print first[3158], corr[3158]

['Xiaodong Zhou', 'Nan Zhang', 'Christina Tan', 'Wolfgang Knoll'] No other authors
['William R. Birch', 'Heow Pueh Lee'] No other authors
['Yang Xiao', 'Xuehong Lu', 'Xinhai Zhang and Chaobin He'] No other authors
No other authors No other authors
No other authors No other authors


In [44]:
corr[2034] = [first[2034][-1]]
corr[2051] = [first[2051][-1]]
corr[2164] = [first[2164][-1]]

In [45]:
first[2034].pop()
first[2051].pop()
first[2164].pop()

'Xinhai Zhang and Chaobin He'

In [47]:
nonFirst[2034] = first[2034][1:]
nonFirst[2164] = first[2164][1:]
nonFirst[2164].append(corr[2164][0].split(' and ')[0])

print nonFirst[2034], nonFirst[2051], nonFirst[2164]

['Nan Zhang', 'Christina Tan'] ['Shi Feng Guo', 'Heow Pueh Lee', 'Kuan Chun Chaw', 'JasonMiklas', 'Serena LayMing Teo', 'Gary H. Dickinson', 'William R. Birch and Boo Cheong Khoo'] ['Xuehong Lu', 'Xinhai Zhang']


In [48]:
first[2034] = [first[2034][0]]
first[2051] = [first[2051][0]]
first[2164] = [first[2164][0]]

corr[2164] = [corr[2164][0].split(' and ')[1]]

print first[2034]
print nonFirst[2034]
print corr[2034]
print '\n'

print first[2051]
print nonFirst[2051]
print corr[2051]
print '\n'

print first[2164]
print nonFirst[2164]
print corr[2164]



['Xiaodong Zhou']
['Nan Zhang', 'Christina Tan']
['Wolfgang Knoll']


['William R. Birch']
['Shi Feng Guo', 'Heow Pueh Lee', 'Kuan Chun Chaw', 'JasonMiklas', 'Serena LayMing Teo', 'Gary H. Dickinson', 'William R. Birch and Boo Cheong Khoo']
['Heow Pueh Lee']


['Yang Xiao']
['Xuehong Lu', 'Xinhai Zhang']
['Chaobin He']


In [145]:
## Clean up the affiliation columns for these three special rows
for i in [2034, 2051, 2164]:
    print PRF.First_Author_Organisation.iloc[i]
    print PRF.Corresponding_Authors_Organisations.iloc[i]

IMRE  
UNKNOWN
IMRE, NUS  
UNKNOWN
Yang Xiao, Xuehong Lu, Xinhai Zhang and Chaobin He  
UNKNOWN


In [146]:
# Since we want the affiliation columns, where first/corr authors is an IMRE staff, to reflect 'IMRE',
# we need to modify row 2164 (both are IMRE staff)

PRF.First_Author_Organisation.iloc[2164] = 'IMRE'
PRF.Corresponding_Authors_Organisations.iloc[2164] = 'IMRE'

/Users/yingjiang/miniconda2/lib/python2.7/site-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [147]:
PRF.iloc[2164, :]

Title_of_Paper                         Synthesis and optical characteristics of organ...
Journal_Title                                         Journal of Nanoparticle Research  
Publication_Release_Number                                                   MPMC07139  
Dept_or_Prog_                                                                       SI  
Significance_of_Paper                                                              ---  
Publication_Date                                                                    2010
First_Author                           Yang Xiao, Xuehong Lu, Xinhai Zhang and Chaobi...
First_Author_Organisation                                                           IMRE
Other_Authors                                                                        NaN
Other_Authors_organisation                                                           NaN
Corresponding_Authors                                                            UNKNOWN
Corresponding_Authors

### Remove papers NEITHER of whose first and corr authors is an IMRE staff.


In [149]:
neitherIMRE = []
for ind, i in enumerate(PRF.First_Author_Organisation):
    if 'IMRE' not in i:
        if 'IMRE' not in PRF.Corresponding_Authors_Organisations.iloc[ind]:
            neitherIMRE.append(ind)
print neitherIMRE

[5, 6, 12, 13, 14, 17, 21, 22, 26, 30, 36, 38, 40, 43, 44, 46, 53, 54, 55, 56, 57, 58, 65, 66, 67, 70, 74, 79, 82, 84, 87, 90, 91, 99, 100, 101, 102, 109, 112, 124, 125, 128, 129, 130, 131, 132, 135, 144, 146, 147, 148, 149, 150, 155, 159, 160, 163, 164, 166, 167, 171, 172, 179, 181, 186, 188, 189, 190, 197, 198, 199, 200, 201, 202, 204, 205, 206, 210, 211, 229, 240, 243, 246, 247, 248, 256, 267, 268, 272, 273, 280, 285, 287, 289, 297, 299, 303, 304, 309, 310, 313, 314, 315, 317, 321, 327, 332, 334, 336, 337, 338, 340, 342, 345, 346, 350, 358, 362, 363, 368, 369, 370, 371, 378, 383, 385, 386, 387, 392, 393, 394, 395, 397, 404, 405, 408, 409, 411, 412, 413, 414, 416, 417, 418, 421, 423, 424, 426, 431, 440, 448, 450, 458, 460, 464, 466, 467, 468, 472, 476, 477, 480, 485, 486, 489, 490, 491, 494, 496, 504, 507, 508, 509, 510, 511, 515, 516, 518, 525, 526, 530, 532, 533, 535, 540, 541, 543, 544, 545, 546, 547, 552, 554, 555, 556, 557, 562, 565, 566, 568, 572, 574, 575, 577, 582, 584, 585, 

In [211]:
'''
# Another way of getting the list of non-IMRE index:
neitherIMRE2 = []
for i in range(3313):
    if ('IMRE' not in PRF.First_Author_Organisation.values[i]) & ('IMRE' not in PRF.Corresponding_Authors_Organisations.values[i]):
        neitherIMRE2.append(i)

print len(neitherIMRE)
print len(neitherIMRE2)
print neitherIMRE == neitherIMRE2'''

1351
1351
True


#### Detailed inspection of authorship affiliations

In [160]:
counter = 0
for i in PRF.First_Author_Organisation.values:
    if 'IMRE' not in i:
        counter +=1
counter

1838

In [ ]:
# There are 1838 papers not written by IMRE first authors.
# Out of these, 1351 do not include an IMRE corr author as well. All IMRE authors are under 'Other_Authors'.

In [190]:
counter = 0
for ind in neitherIMRE:
    if PRF.Corresponding_Authors_Organisations.values[ind] == 'UNKNOWN':
        counter +=1
print counter

660


In [ ]:
# There are 660 papers where the corr author MAY BE an IMRE staff, but is buried in Other_Authors, and cannot be determined.

In [191]:
for ind in neitherIMRE:
    if PRF.Corresponding_Authors_Organisations.values[ind] == 'UNKNOWN':
        print ind
        print nonFirst[ind]
        print PRF.Other_Authors_organisation.values[ind]
        print 'IMRE' in PRF.Other_Authors_organisation.values[ind].strip()[-4:]
        print '\n'

1918
['Tjiu Weng Weei', 'Liu Tianxi', 'Lui Wai Yi', 'Phang In Yee', 'Zhang Wei-De']
ASTAR, Inst Mat Res & Engn, Singapore 117602, Singapore; Fudan Univ, Key Lab Mol Engn Polymers, Minist Educ, Dept Macromol Sci, Shanghai 200433, Peoples R  
False


1990
['Liu Bin; Low Edwin Kim Fong; Liu Wei; Yang Mingchu; Tan Leng-Seow; Teo Kie Leong; Yeo Yee-Chia']
IMRE and NUS 
False


1993
['Andrew T. S. Wee', 'Cedric Troadec', 'and N. Chandrasekhar']
NUS, IMRE 
True


1998
['E. J. Teo']
Institute of Materials Research and Engineering  
False


1999
['JS Pan', 'WT Zheng', 'J Zhou', 'CQ. Sun']
IMRE, JLU, NTU 
False


2006
['Choon Wee Kee', 'Shenyu Li', 'Hor T S A', 'Jin Zhao']
Chem Dep, NUS, IMRE and NUS, GIST-TUM 
False


2007
['Murugadoss A', 'Hor T S A', 'Sakurai H']
Research Center for Molecular Scale Nanoscience, Institute for Molecular Science, Myodaiji, IMRE and NUS, JSTA 
False


2008
['Koh LL', 'Liu ZP', 'Weng ZQ', 'Hor T S A']
Chem Det, NUS, Fudan Univ, Fuzhou Univ, IMRE and NUS 
False


2

AttributeError: 'float' object has no attribute 'strip'

In [ ]:
# Inspected, for all the UNKNOWN corr rows, whether the last author from Other_Authors is from IMRE.
# If so, the last author may be taken to be the corr.
# The result is that most of them are not. Or a mix of both IMRE and other institutions.
# It's too tedious to extract info from these 660 rows. Treat these as non-relevant papers.

In [152]:
# Create the more relevant dataframe.
PRF_IMRE = PRF.drop(neitherIMRE, axis=0)

In [154]:
PRF_IMRE.head()

Title_of_Paper  \
0  Recent Progress in Chemical Vapor Deposition G...   
1  A Polydopamine Coating Ultralight Graphene Mat...   
2  MOLECULAR DESIGN OF BIOINSPIRED NANOSTRUCTURES...   
3  Biodegradable Thermogelling Polymers for Biome...   
4  Dual-responsive Reversible Photo/Thermogelling...   

                                       Journal_Title  \
0  Progress in Crystal Growth and Characterizatio...   
1                                 Renewable Energy     
2   Journal of Molecular and Engineering Materials     
3                                     MRS Bulletin     
4  Journal of Polymer Science, Part A: Polymer Ch...   

  Publication_Release_Number                  Dept_or_Prog_  \
0               MPC/16-128                           -MPC     
1                AS/16-056    Advanced Energy Storage Lab     
2                BB/16-185       -Biomedical & Biomimetic     
3                CT/16-229      -Consumer Care Technology     
4                CT/16-227      -Consumer Care Technology     

                               Significance_of_Paper Publication_Date  \
0  Reviews the latest progress in CVD growth of T...           Jul-16   
1  The soft polydopmine (PDA) layer with polar fu...           Aug-16   
2  In this review, we present the recent developm...           Aug-16   
3  Thermogelling polymers belong to a class of st...           Aug-16   
4  This work describes the synthesis of a novel p...           Jun-16   

             First_Author First_Author_Organisation  \
0       WONG Swee Liang                      IMRE     
1              Zhou Lan          Fudan University     
2   stu-victorxu (IMRE)                      IMRE     
3  LIOW Sing Shy (IMRE)                      IMRE     
4  DOU Qing Qing (IMRE)                      IMRE     

                                       Other_Authors  \
0                        LIU Hong Fei, CHI Dongzhi     
1  LIU Zhao Lin (IMRE), ZONG Yun (IMRE), Yu Aishui     
2  ZHENG Xinting, Beverly MOK Yin Leng, stu-salwa...   
3     Anis ABDUL KARIM (IMRE), LOH Xian Jun (IMRE)     
4  LIOW Sing Shy (IMRE), LOH Xian Jun (IMRE), Wen...   

       Other_Authors_organisation  \
0                     IMRE, IMRE    
1   IMRE, IMRE, Fudan University    
2   IMRE, IMRE, IMRE, IMRE, IMRE    
3                     IMRE, IMRE    
4  IMRE, IMRE, Xiamen University    

                               Corresponding_Authors  \
0                                      CHI Dongzhi     
1  LIU Zhao Lin (IMRE), ZONG Yun (IMRE), Yu Aishui     
2                                      TAN Yen Nee     
3                              LOH Xian Jun (IMRE)     
4                              LOH Xian Jun (IMRE)     

  Corresponding_Authors_Organisations Project_Finance_Code  \
0                               IMRE      IMRE/15-2C0115     
1       IMRE, IMRE, Fudan University      IMRE/14-1C0243     
2                               IMRE      IMRE/14-8C0439     
3                               IMRE      IMRE/13-2P0806     
4                               IMRE      IMRE/13-2P0806     

                                       Project_Title  Web_ID  
0  Large area growth of atomically thin 2D semico...    4696  
1  High-performance sulfur nanostructured cathode...    4695  
2  Development of highly efficient siRNA-conjugat...    4694  
3      Polymer Bank for Personal Care Applications      4693  
4      Polymer Bank for Personal Care Applications      4692

In [153]:
PRF_IMRE.shape

(1962, 15)

In [319]:
firstIMRE = [i for ind, i in enumerate(first) if ind not in neitherIMRE]
# corrIMRE = [i for ind, i in enumerate(corr) if ind not in neitherIMRE]

In [320]:
print firstIMRE[:20]
print corrIMRE[:20]

[['WONG Swee Liang'], ['Zhou Lan'], ['stuvictorxu'], ['LIOW Sing Shy'], ['DOU Qing Qing'], ['T L Duan'], ['YU Yong'], ['YU Yong'], ['Liu Junyi'], ['David'], ['Stephen John Fox'], ['Ghislaine Barouti'], ['Milos Petrovic'], ['Milos Petrovic'], ['Milos Petrovic'], ['Goutam Kumar DALAPATI'], ['zhang chong jing'], ['Yuan you yong'], ['GUO Shifeng'], ['Jae Yoon Hwang']]
[['CHI Dongzhi'], ['LIU Zhao Lin', 'ZONG Yun', 'Yu Aishui'], ['TAN Yen Nee'], ['LOH Xian Jun'], ['LOH Xian Jun'], ['PAN Ji Sheng', 'D S Ang'], ['TAN Yen Nee'], ['TAN Yen Nee'], ['REN Yi', 'CHIAM Sing Yang', 'Li Fong Yau Sam'], ['LOH Xian Jun'], ['LOH Xian Jun', 'Chandra Shekhar Verma', 'Navin Kumar Verma', 'Rajamani Lakshminarayanan'], ['LOH Xian Jun', 'Sophie M Guillaume'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrisha'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'], ['Goutam Kumar DALAPATI'], ['LIU Bin'], ['LIU Bin'], ['Dominik Janczewski', 'G Julius Vancso'], ['LI Zibia

In [72]:
## Clean up multiple-author rows

# first author:
# - Look at the multiple-first-author cases
# - Remove non alphabet characters
# - Solve the abbreviations problem
# - Comparing with staff list, normalize lower cases
# - Comparing with staff list, normalize abbreviations
 

# corr author:
# - Look at the multiple-first-author cases
# - Remove non alphabet characters
# - Solve the abbreviations problem
# - Identify those who are IMRE corr authors; remove the rest

'''
# For 'first', almost all of the length-2 author lists are actually a single author (ie there are very few multiple first author cases).
# Correct the formatting.
for ind, i in enumerate(first):
    if len(i) > 1:
        print ind
        print i
        tmp = [' '.join(i).strip()] # Join the 'non-existent' 2nd entry with the first
        first[ind] = [re.sub("[^a-zA-Z\s]", "", tmp[0])] # Strip of any non-alphabet and non-space character
        print first[ind]

## Repeat the stripping of non-alphabet for all first authors
for ind, i in enumerate(first):
    first[ind] = [re.sub("[^a-zA-Z\s]", "", first[ind][0])]

# Rows 2312 and 3158 got changed but it's ok. They're gonna be removed anyway.
'''

'''
print first
# Looks much cleaner
'''

In [321]:
for ind, i in enumerate(firstIMRE):
    firstIMRE[ind] = [re.sub("[^a-zA-Z\s]", "", firstIMRE[ind][0])]
print firstIMRE

[['WONG Swee Liang'], ['Zhou Lan'], ['stuvictorxu'], ['LIOW Sing Shy'], ['DOU Qing Qing'], ['T L Duan'], ['YU Yong'], ['YU Yong'], ['Liu Junyi'], ['David'], ['Stephen John Fox'], ['Ghislaine Barouti'], ['Milos Petrovic'], ['Milos Petrovic'], ['Milos Petrovic'], ['Goutam Kumar DALAPATI'], ['zhang chong jing'], ['Yuan you yong'], ['GUO Shifeng'], ['Jae Yoon Hwang'], ['Roozbeh Siavash Moakhar'], ['LI Haidong'], ['HE Jiating'], ['LIU Rong Rong'], ['WAi Hoe Tham'], ['WANG Shengqin'], ['LIU Hongwei'], ['Miao Qi'], ['David PARAMELLE'], ['YANG Ming'], ['Avishek Das'], ['Neo Wei Teng'], ['YE Qun'], ['Ri Rong'], ['Bo He'], ['Zhou Lan'], ['LEE Chee Wei'], ['Pengfei Li'], ['TAN Mein Jin'], ['REN Wei'], ['CHAN Qi Yu Benjamin'], ['KAI Dan'], ['Goutam Kumar DALAPATI'], ['JIANG Lu'], ['Szymon Godlewski'], ['GOH Xiao Ming'], ['LIU Hong Fei'], ['LI Zibiao'], ['GUO Shifeng'], ['Wei Liu'], ['Davide Ansovini'], ['Afriyanti SUMBOJA'], ['LIU Hong Fei'], ['HUANG Yuli'], ['Wang Zhijuan'], ['Tingting Yin'], ['L

In [255]:
## Abbreviations of both first and corr:
# Change the 5 format below into a single one:
# - AB Surname
# - A B Surname
# - Surname AB
# - Surname A B
# - A Surname B

# Break the name into words
# Check if each word is a "glued-up acronym", e.g. AB or ABC
# Ways to check:
# - Whether it's of length 2 or 3
# - Whether it's all upper letters
# - Whether it contains no vowels. Especially adjacent consonants:
# - CC
# - CCC
# - CCO
# - OCC
# (CCO doesn't work because it brings up surnames such as Chi, Zhu, etc. OCC brings up OWH. Use only pure consonants.)
# The goal is to break 'AB' up into 'A B', and then move it to the back of the name.


In [13]:
'''
for ind, i in enumerate(firstIMRE):
#     print ind
    for j in i[0].split(' '):
        if ((len(j) == 2)|(len(j) == 3)) & (j.upper() == j) & (j != 'NG'):
            consonants = [not i in 'aeiouAEIOU' for i in j]
            if (consonants == [True, True]) | (consonants == [True, True, True]):
                print ind
                print firstIMRE[ind]
                print j
'''
'''
for ind, i in enumerate(firstIMRE):
    tmp = []
    for ind2, j in enumerate(i[0].split(' ')):
        if ((len(j) == 2)|(len(j) == 3)) & (j.upper() == j) & (j != 'NG'):
            consonants = [not letter in 'aeiouAEIOU' for letter in j]
            if (consonants == [True, True]) | (consonants == [True, True, True]):
                print ind
                print firstIMRE[ind]
                tmp.append(' '.join(j))
        else:
            tmp.append(j)
    firstIMRE[ind] = ' '.join(tmp)              

'''

def breakAbbreviations(vector = firstIMRE):
    # vector is a list of lists.
    # Each list contains a single name.
    for ind, i in enumerate(vector):
        tmp = []
        for ind2, j in enumerate(i[0].split(' ')):
            if ((len(j) == 2)|(len(j) == 3)) & (j.upper() == j) & (j != 'NG'):
                consonants = [not letter in 'aeiouAEIOU' for letter in j]
                if (consonants == [True, True]) | (consonants == [True, True, True]):
                    tmp.append(' '.join(j))
                else:
                    tmp.append(j)
            else:
                tmp.append(j)
        vector[ind] = [' '.join(tmp)]
    return vector

def breakAbbreviations1(vector = corr[1]):
    # vector is a list of names.
    for ind, i in enumerate(vector): # each name
        tmp = []
        for ind2, j in enumerate(i.split(' ')): # each character
            if ((len(j) == 2)|(len(j) == 3)) & (j.upper() == j) & (j != 'NG'):
                consonants = [not letter in 'aeiouAEIOU' for letter in j]
                if (consonants == [True, True]) | (consonants == [True, True, True]):
                    tmp.append(' '.join(j))
                else:
                    tmp.append(j)
            else:
                tmp.append(j)
        vector[ind] = ' '.join(tmp)
    return vector

NameError: name 'firstIMRE' is not defined

In [335]:
firstIMRE_break = breakAbbreviations(firstIMRE)
print firstIMRE_break

[['WONG Swee Liang'], ['Zhou Lan'], ['stuvictorxu'], ['LIOW Sing Shy'], ['DOU Qing Qing'], ['T L Duan'], ['YU Yong'], ['YU Yong'], ['Liu Junyi'], ['David'], ['Stephen John Fox'], ['Ghislaine Barouti'], ['Milos Petrovic'], ['Milos Petrovic'], ['Milos Petrovic'], ['Goutam Kumar DALAPATI'], ['zhang chong jing'], ['Yuan you yong'], ['GUO Shifeng'], ['Jae Yoon Hwang'], ['Roozbeh Siavash Moakhar'], ['LI Haidong'], ['HE Jiating'], ['LIU Rong Rong'], ['WAi Hoe Tham'], ['WANG Shengqin'], ['LIU Hongwei'], ['Miao Qi'], ['David PARAMELLE'], ['YANG Ming'], ['Avishek Das'], ['Neo Wei Teng'], ['YE Qun'], ['Ri Rong'], ['Bo He'], ['Zhou Lan'], ['LEE Chee Wei'], ['Pengfei Li'], ['TAN Mein Jin'], ['REN Wei'], ['CHAN Qi Yu Benjamin'], ['KAI Dan'], ['Goutam Kumar DALAPATI'], ['JIANG Lu'], ['Szymon Godlewski'], ['GOH Xiao Ming'], ['LIU Hong Fei'], ['LI Zibiao'], ['GUO Shifeng'], ['Wei Liu'], ['Davide Ansovini'], ['Afriyanti SUMBOJA'], ['LIU Hong Fei'], ['HUANG Yuli'], ['Wang Zhijuan'], ['Tingting Yin'], ['L

In [522]:
## Bring staff name dataset in to compare lower cases and abbreviations
## First, compare directly

firstInRecord = []
for i in firstIMRE_break:
#     print i[0]
    if i[0] in staff.Staff_Name.values:
        firstInRecord.append(i)
print len(firstIMRE_break)
print len(firstInRecord)
# 417 of the first authors are confirmed IMRE staff.

1962
417


In [421]:
## THen compare lower cases
staffNameLower = []
for ind, val in staff.Staff_Name.iteritems():
    staffNameLower.append(val.lower())
len(staffNameLower)    

293

In [523]:
indLower = []
for ind, i in enumerate(firstIMRE_break):
    if i[0].lower() in staffNameLower:
        indLower.append(ind)
        
print len(firstIMRE)
print len(indLower)
# Comparing lower cases captures 67 more.

1962
483


In [525]:
## Compare cases where surname and given names are flipped
indOrder = []
for ind, i in enumerate(firstIMRE_break):
    nameSplit = i[0].split(' ')
    lastChar = nameSplit[-1]
    nameSplit.pop()
    firstChars = ' '.join(nameSplit)
    if ' '.join((lastChar, firstChars)).lower() in staffNameLower:
        indOrder.append(ind)    
#         print ind
#         print i

print len(indOrder)
# This adds another 19 cases, bringing the total to 483+19 = 502

19


In [526]:
## Finally compare abbrevs
firstIMRE_Abbrev = [i for i in firstIMRE_break if (firstIMRE_break.index(i) not in indLower) & (firstIMRE_break.index(i) not in indOrder)]
print len(firstIMRE_Abbrev)
print firstIMRE_Abbrev

1460
[['Zhou Lan'], ['stuvictorxu'], ['T L Duan'], ['Liu Junyi'], ['David'], ['Stephen John Fox'], ['Ghislaine Barouti'], ['Milos Petrovic'], ['Milos Petrovic'], ['Milos Petrovic'], ['zhang chong jing'], ['Yuan you yong'], ['Jae Yoon Hwang'], ['Roozbeh Siavash Moakhar'], ['LI Haidong'], ['WAi Hoe Tham'], ['Miao Qi'], ['Avishek Das'], ['Neo Wei Teng'], ['Ri Rong'], ['Bo He'], ['Zhou Lan'], ['LEE Chee Wei'], ['Pengfei Li'], ['CHAN Qi Yu Benjamin'], ['Szymon Godlewski'], ['Wei Liu'], ['Davide Ansovini'], ['Wang Zhijuan'], ['Tingting Yin'], ['LUO Jie'], ['Zhuo Wang'], ['Xiaowei Dou'], ['LAY Chee leng'], ['Wang Yumei'], ['Teo Wei Jie'], ['Soumyajit Das'], ['Zeng Wangdong'], ['Y Ni'], ['P Hu'], ['Qi QingBiao'], ['LUO Jie'], ['Wei Du'], ['SHI Zugui'], ['Terence K S Wong'], ['Yonggang Xiang'], ['Li Hui'], ['LEONG Wei Lin'], ['Roozbeh Siavash Moakhar'], ['Hu Jue'], ['Saeid MasudyPanah'], ['ZHU Xiaoying'], ['Daria B Trushina'], ['Avishek Das'], ['Zifeng Zhang'], ['Shen Zhiyuan'], ['Yumeng Yang']

It's to be noted that the remaining names (1962-483 = 1499) consist of
- Non-IMRE staff
- Ex-IMRE staff
- a small percentage of staff with abbreviated names

In [521]:
staffSurname = []
staffGivenName = [] # This will be a list of given name characters (in separate words).
for ind, val in staff.Staff_Name.iteritems():
    nameWords = val.split(' ')
#     print nameWords
    for ind2, word in enumerate(nameWords):
#         print ind2, word
        if word.upper() == word:
            staffSurname.append(word)
            del nameWords[ind2]
            staffGivenName.append(nameWords)
#         else:
#             staffSurname.append('Not clear')
#             staffGivenName.append(nameWords)

print staffSurname
print staffGivenName

['SUMBOJA', 'BRZOZOWSKA', 'KUSHWAHA', 'AN', 'NG', 'TAN', 'ABDUL', 'SADOVOY', 'CHEONG', 'KYAW', 'BAI', 'MOK', 'ONG', 'MANZANO', 'TROADEC', 'CHAI', 'CHAN', 'CHAN', 'CHEE', 'CHEE', 'CHELLAPPAN', 'CHEN', 'CHEN', 'CHEN', 'CHEN', 'CHEN', 'CHENG', 'CHENG', 'CHEONG', 'CHEW', 'CHIAM', 'CHIEN', 'CHO', 'CHONG', 'TAN', 'CHUA', 'LIU', 'LEE', 'PARAMELLE', 'SENG', 'WUU', 'DENG', 'TAN', 'FAM', 'DING', 'DING', 'DONG', 'ZHOU', 'LAI', 'DOU', 'TANG', 'WILLIAMS', 'GE', 'GOH', 'GOH', 'GOH', 'GOH', 'SUBRAMANIAN', 'DALAPATI', 'GOH', 'GU', 'GUO', 'HAN', 'LIM', 'HE', 'HE', 'HE', 'TANOTO', 'GOH', 'KAWAI', 'HO', 'HONG', 'HUANG', 'HUANG', 'HUANG', 'HUANG', 'HUI', 'TEE', 'WONG', 'KUMAR', 'YEO', 'TOH', 'JIANG', 'JIANG', 'JIANG', 'JIN', 'YANG', 'GOH', 'ONG', 'KAI', 'CHONG', 'KE', 'KE', 'HIPPALGAONKAR', 'KHIN', 'KHIN', 'KOH', 'KOH', 'KONG', 'KUANG', 'KWOK', 'LAI', 'BERA', 'TAN', 'LAU', 'SUTARLIE', 'LE', 'LEE', 'LEE', 'LEE', 'LEONG', 'LI', 'LI', 'LI', 'LI', 'LI', 'LI', 'LIAN', 'LIEW', 'LIEW', 'LIM', 'LIM', 'LIM', 'LIM'

In [ ]:
'''# INCOMPLETE....
for ind, i in firstIMRE_Abbrev:
    nameSplit = i.split(' ')
    for ind2, j in nameSplit:
        if len(j) == 1:
            grab the first characters of the split staffGivenName list
            Compare these characters with j
        else:
            j.upper() in staffSurname'''

In [368]:
## Overall inspection of names not found in namelist
for ind, i in enumerate(firstIMRE):
    if i[0] not in staff.Staff_Name.values:
        print i[0]

Zhou Lan
stuvictorxu
T L Duan
Liu Junyi
David
Stephen John Fox
Ghislaine Barouti
Milos Petrovic
Milos Petrovic
Milos Petrovic
zhang chong jing
Yuan you yong
Jae Yoon Hwang
Roozbeh Siavash Moakhar
LI Haidong
WAi Hoe Tham
Miao Qi
Avishek Das
Neo Wei Teng
Ri Rong
Bo He
Zhou Lan
LEE Chee Wei
Pengfei Li
CHAN Qi Yu Benjamin
Szymon Godlewski
Wei Liu
Davide Ansovini
Wang Zhijuan
Tingting Yin
LUO Jie
Zhuo Wang
Xiaowei Dou
LAY Chee leng
Wang Yumei
Teo Wei Jie
Soumyajit Das
Zeng Wangdong
Y Ni
P Hu
Qi QingBiao
LUO Jie
Wei Du
SHI Zugui
Terence K S Wong
Yonggang Xiang
Li Hui
LEONG Wei Lin
Roozbeh Siavash Moakhar
Hu Jue
Saeid MasudyPanah
ZHU Xiaoying
Daria B Trushina
Avishek Das
Zifeng Zhang
Shen Zhiyuan
Yumeng Yang
GENG Dongsheng
GUO Hong Chen
Xiao Zhao
W H Tham
Saeid MasudyPanah
Roger M Pallares
JIN Guorui
Lim Zhenglong
Kenny Low Zhi Wei
Su Xinyi
yuan youyong
yuan youyong
liang jing
yuan youyong
yuan youyong
feng guangxue
yuan you yong
Yuan Youyong
SU Xiao Di
Wu Yunlong
JIANG Shan
Ray NG Jia Hong
S

In [398]:
for ind, i in enumerate(firstIMRE):
    if 'Cally' in i[0]:
        print ind

191


Upon comparing the first author names and staff namelist, it appears the latter is not updated. :(

Use the first authors from lower-case and order indices first. (502)

## Normalize corr author names (challenging)

#### Remove special characters

In [251]:
for ind, i in enumerate(corrIMRE):
    print ind
    print i
    tmp = []
    for ind2, j in enumerate(i):
        print ind2
        print j
        corrIMRE[ind][ind2] = re.sub("[^a-zA-Z\s]", "", j)
        print corrIMRE[ind][ind2]
        print '\n'


0
['CHI Dongzhi']
0
CHI Dongzhi
CHI Dongzhi


1
['LIU Zhao Lin', 'ZONG Yun', 'Yu Aishui']
0
LIU Zhao Lin
LIU Zhao Lin


1
ZONG Yun
ZONG Yun


2
Yu Aishui
Yu Aishui


2
['TAN Yen Nee']
0
TAN Yen Nee
TAN Yen Nee


3
['LOH Xian Jun']
0
LOH Xian Jun
LOH Xian Jun


4
['LOH Xian Jun']
0
LOH Xian Jun
LOH Xian Jun


5
['PAN Ji Sheng', 'D. S. Ang']
0
PAN Ji Sheng
PAN Ji Sheng


1
D. S. Ang
D S Ang


6
['TAN Yen Nee']
0
TAN Yen Nee
TAN Yen Nee


7
['TAN Yen Nee']
0
TAN Yen Nee
TAN Yen Nee


8
['REN Yi', 'CHIAM Sing Yang', 'Li Fong Yau Sam']
0
REN Yi
REN Yi


1
CHIAM Sing Yang
CHIAM Sing Yang


2
Li Fong Yau Sam
Li Fong Yau Sam


9
['LOH Xian Jun']
0
LOH Xian Jun
LOH Xian Jun


10
['LOH Xian Jun', 'Chandra Shekhar Verma', 'Navin Kumar Verma', 'Rajamani Lakshminarayanan']
0
LOH Xian Jun
LOH Xian Jun


1
Chandra Shekhar Verma
Chandra Shekhar Verma


2
Navin Kumar Verma
Navin Kumar Verma


3
Rajamani Lakshminarayanan
Rajamani Lakshminarayanan


11
['LOH Xian Jun', 'Sophie M. Guillaume']
0
LOH Xian J

In [253]:
corrIMRE

[['CHI Dongzhi'],
 ['LIU Zhao Lin', 'ZONG Yun', 'Yu Aishui'],
 ['TAN Yen Nee'],
 ['LOH Xian Jun'],
 ['LOH Xian Jun'],
 ['PAN Ji Sheng', 'D S Ang'],
 ['TAN Yen Nee'],
 ['TAN Yen Nee'],
 ['REN Yi', 'CHIAM Sing Yang', 'Li Fong Yau Sam'],
 ['LOH Xian Jun'],
 ['LOH Xian Jun',
  'Chandra Shekhar Verma',
  'Navin Kumar Verma',
  'Rajamani Lakshminarayanan'],
 ['LOH Xian Jun', 'Sophie M Guillaume'],
 ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrisha'],
 ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'],
 ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'],
 ['Goutam Kumar DALAPATI'],
 ['LIU Bin'],
 ['LIU Bin'],
 ['Dominik Janczewski', 'G Julius Vancso'],
 ['LI Zibiao', 'LOH Xian Jun'],
 ['Gregory GOH Kia Liang'],
 ['LI Xu', 'LIU Hong'],
 ['LI Xu'],
 ['Nikodem TOMCZAK'],
 ['TRIPATHY Sudhiranjan', 'Lakshmi Kanta BERA', 'Diing Shenp Ang'],
 ['WANG Shengqin', 'Yingxi Zhu'],
 ['Lu Junpeng', 'SOW Chorng Haur'],
 ['David J Young', 'T S Andy Hor', 'ZHANG Zhongxing'],
 ['Jatin Nitin KUMAR', 'LIU Ye'],
 ['YANG 

#### Break abbreviations

In [254]:
for ind, i in enumerate(corrIMRE):
    if len(i) > 1:
        print ind
        print i
        print PRF_IMRE.Corresponding_Authors_Organisations.values[ind]

# Challenges:
    # Multiple commas, inability to demarcate affiliations.
    # Old authors who were in IMRE are no longer in IMRE (Chen Zhi-kuan, Andy Hor).
    # Authors who weren't in IMRE but who joined IMRE later.
# Strategy:
    # For those corr Author that contain IMRE, split into individual authors
    # Remove non-alphabets
    # Compare with staff list (while solving the abbreviation problem)

1
['LIU Zhao Lin', 'ZONG Yun', 'Yu Aishui']
IMRE, IMRE, Fudan University 
5
['PAN Ji Sheng', 'D S Ang']
IMRE, NTU 
8
['REN Yi', 'CHIAM Sing Yang', 'Li Fong Yau Sam']
IMRE, IMRE, National University of Singapore 
10
['LOH Xian Jun', 'Chandra Shekhar Verma', 'Navin Kumar Verma', 'Rajamani Lakshminarayanan']
IMRE, BII, NTU, SERI 
11
['LOH Xian Jun', 'Sophie M Guillaume']
IMRE, Institut des Sciences Chimiques de Rennes, UMR 6226 CNRS - Universit̩ de Rennes 1 
12
['CHELLAPPAN Vijila', 'Prof Seeram Ramakrisha']
IMRE, Department of Mechanical Engineering, National University of Singapore, Singapore 117576. 
13
['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna']
IMRE, Department of Mechanical Engineering, National University of Singapore, Singapore 117576 
14
['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna']
IMRE, Department of Mechanical Engineering, National University of Singapore, Singapore 117576 
18
['Dominik Janczewski', 'G Julius Vancso']
Warsaw University of Technology, University of Twen

In [359]:
corrIMRE_break = []
for ind, allCorr in enumerate(corrIMRE):
#     print allCorr
    corrIMRE_break.append(breakAbbreviations1(allCorr))
print corrIMRE_break

[['CHI Dongzhi'], ['LIU Zhao Lin', 'ZONG Yun', 'Yu Aishui'], ['TAN Yen Nee'], ['LOH Xian Jun'], ['LOH Xian Jun'], ['PAN Ji Sheng', 'D S Ang'], ['TAN Yen Nee'], ['TAN Yen Nee'], ['REN Yi', 'CHIAM Sing Yang', 'Li Fong Yau Sam'], ['LOH Xian Jun'], ['LOH Xian Jun', 'Chandra Shekhar Verma', 'Navin Kumar Verma', 'Rajamani Lakshminarayanan'], ['LOH Xian Jun', 'Sophie M Guillaume'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrisha'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'], ['CHELLAPPAN Vijila', 'Prof Seeram Ramakrishna'], ['Goutam Kumar DALAPATI'], ['LIU Bin'], ['LIU Bin'], ['Dominik Janczewski', 'G Julius Vancso'], ['LI Zibiao', 'LOH Xian Jun'], ['Gregory GOH Kia Liang'], ['LI Xu', 'LIU Hong'], ['LI Xu'], ['Nikodem TOMCZAK'], ['TRIPATHY Sudhiranjan', 'Lakshmi Kanta BERA', 'Diing Shenp Ang'], ['WANG Shengqin', 'Yingxi Zhu'], ['Lu Junpeng', 'SOW Chorng Haur'], ['David J Young', 'T S Andy Hor', 'ZHANG Zhongxing'], ['Jatin Nitin KUMAR', 'LIU Ye'], ['YANG Ming', 'WANG Shi Jie'], ['Goutam Ku

Next, revisit affiliations. We know the corresponding authors' affiliations are messy and hard to tell from the PRF dataset alone. Even the first authors containing 'IMRE' as the keyword in affiliations may not be truly affiliated:

In [360]:
# Some first authors have affiliation secondary to IMRE.
for ind, i in enumerate(PRF.First_Author_Organisation):
    if (',' in i) & ('IMRE' in i):
        print ind
        print 'Secondary affiliation: ', i

94
Secondary affiliation:  NTU, CBC; IMRE  
251
Secondary affiliation:  IHPC, and attached to IMRE  
288
Secondary affiliation:  NUS, IMRE student  
325
Secondary affiliation:  NTU, CBC; IMRE  
366
Secondary affiliation:  IMRE and NUS (out station, in US for two years)  
579
Secondary affiliation:  former IMRE staff, now studying in UK  
587
Secondary affiliation:  NUS, IMRE student  
685
Secondary affiliation:  SUTD, NTU, IMRE  
950
Secondary affiliation:  CBC, NTU; IMRE, A*Star  
956
Secondary affiliation:  IHPC, attached to IMRE  
974
Secondary affiliation:  NUS, IMRE PhD student  
1265
Secondary affiliation:  NUS, IMRE PhD student  
1381
Secondary affiliation:  NTU, IMRE  
1654
Secondary affiliation:  IMRE, but currently in Stanford on postdoctoral attachment  
1659
Secondary affiliation:  Previous IMRE STAFF, now with P&G  
1811
Secondary affiliation:  IMRE, Singapore  
1812
Secondary affiliation:  IMRE, Singapore  
1893
Secondary affiliation:  IMRE,University of Waterloo  
1970
S

We need to compare with the current staff namelist to extract the real IMRE authors.

Let's do this with first authors for now, because it's a single-name list and is easier to handle.

We will also stick to the 502 first authors whom we know for sure are IMRE staff.

In [527]:
indUseful = indLower + indOrder
indUseful.sort()
print indUseful

[0, 3, 4, 6, 7, 15, 18, 22, 23, 25, 26, 28, 29, 32, 38, 39, 41, 42, 43, 45, 46, 47, 48, 51, 52, 53, 57, 59, 63, 64, 71, 75, 78, 81, 82, 84, 86, 87, 89, 93, 94, 95, 96, 98, 101, 102, 103, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 126, 127, 128, 129, 130, 147, 148, 154, 156, 162, 163, 168, 180, 183, 186, 188, 189, 193, 196, 197, 198, 199, 200, 202, 203, 209, 210, 211, 212, 213, 214, 215, 216, 221, 222, 223, 226, 228, 229, 232, 234, 239, 241, 243, 245, 251, 253, 257, 261, 262, 263, 264, 265, 266, 268, 269, 270, 271, 272, 274, 275, 276, 279, 280, 281, 282, 285, 286, 287, 288, 290, 293, 298, 301, 303, 305, 306, 307, 309, 310, 314, 315, 317, 323, 324, 325, 337, 340, 341, 344, 345, 348, 349, 351, 355, 356, 357, 358, 360, 365, 369, 377, 378, 381, 389, 392, 393, 399, 401, 403, 404, 405, 407, 416, 419, 421, 422, 423, 424, 425, 427, 430, 431, 432, 443, 444, 447, 448, 449, 450, 451, 452, 453, 456, 457, 467, 470, 471, 472, 473, 474, 475, 478, 479, 482, 483, 484, 485

In [529]:
PRF_IMRE2 = PRF_IMRE.iloc[indUseful, :]

In [530]:
PRF_IMRE2.shape

(502, 15)

In [537]:
PRF_IMRE2.reset_index()

index                                     Title_of_Paper  \
0        0  Recent Progress in Chemical Vapor Deposition G...   
1        3  Biodegradable Thermogelling Polymers for Biome...   
2        4  Dual-responsive Reversible Photo/Thermogelling...   
3        8  Rational Design of Biomolecular Templates for ...   
4        9  Bovine Serum Albulmin Protein-Templated Silver...   
5       23  All earth abundant materials for low cost sola...   
6       27  Measuring protein isoelectric points by AFM fo...   
7       32  Controlled Growth of Metal-Organic Framework o...   
8       33  Transparent hydrophobic antimicrobial coatings...   
9       35  Dielectrophoresis Directed Nanocolloidal and S...   
10      37  Ultrahigh photoconductivity of bandgap-graded ...   
11      41  Photothermally responsive gold nanoparticles c...   
12      42  Interfacial Interaction between HfO2 and MoS2:...   
13      48  Cubic Polyhedral Oligomeric Silsesquioxane Bas...   
14      60  Bio-degradable Electronics: Cornerstone for su...   
15      61  Dodecylated lignin-g-PLA for effective toughen...   
16      63  Engineering poly(lactide)-lignin nanofibers wi...   
17      64  Defect analysis and performance evaluation of ...   
18      68  Novel MII (M = Mn, Fe, Co, Ni) Coordination As...   
19      71  Comparative Study of Plasmonic Colors from All...   
20      72  High-resolution X-ray diffraction and micro-Ra...   
21      73  Recent Advances in Stereocomplexation of Enant...   
22      75  Method and Analysis for Determining Yielding o...   
23      78  Progress in Development of Flexible Metal-Air ...   
24      80  Droplets induced dot, dot-in-hole, and hole st...   
25      81  Gap States at Low-Angle Grain Boundaries in Mo...   
26      88  Easy fabrication of high quality nickel mold f...   
27      92  Thermally Stable and Sterilizable Polymer Tran...   
28      96  Effect of morphological and structural propert...   
29      97  Re(I) Carbonyl Complexes containing Pyridyl-Im...   
..     ...                                                ...   
472   2004  Some recent developments of polyhedral oligome...   
473   2005  Self-Assembly and Micellization of a Dual Ther...   
474   2017  photoelectrochemical cell with carbon nanotubes     
475   2044  Organic���inorganic hybrid liquid crystals der...   
476   2058  Physical and electrochemical characterizations...   
477   2070  Efficient optimization on DSSC with low freque...   
478   2128  Impact of vitronectin concentration and surfac...   
479   2149  Morphological tuning, self-assembly and optica...   
480   2151  Efficient gene delivery with paclitaxel-loaded...   
481   2157  A generic approach for preparing core-shell ca...   
482   2164  Synthesis and optical characteristics of organ...   
483   2193  Large strain and high energy storage density i...   
484   2233  Conductance decay of a surface hydrogen tunnel...   
485   2256  Direct transfer of gold nanoislands from a MoS...   
486   2272  Direct imprinting of high resolution TiO2 nano...   
487   2274  Structural and Electrical Characterization of ...   
488   2300  Morphologies and Electrical Properties of Elec...   
489   2309  Interfacial and mechanical studies of a compos...   
490   2347  Spin-coatable HfO2 resist for optical and elec...   
491   2490  Thermally Responsive Hydrogel Film Studied by ...   
492   2498  Preface to special topic, Selected Papers from...   
493   2567  DNA assembly on streptavidin modified surface:...   
494   2647  Composite Nafion� membrane embedded with hybri...   
495   2656  Nature and dynamics of photoexcited states in ...   
496   2692  Charge Transport Studies in Electroluminescent...   
497   2699  Greatly Enhanced Optical Properties of ZnO Nan...   
498   2712  The influence of nitrogen plasma treatment on ...   
499   2891  A study of interface-sustained ferromagnetism ...   
500   3050  A study of charge transport in a novel electro...   
501   3175  Non-dispersive hole transpor

# Extract keywords

## Using chunking

In [542]:
tritext = PRF_IMRE2.Title_of_Paper.iloc[1] + PRF_IMRE2.Significance_of_Paper.iloc[1]

In [ ]:
def extract_candidate_chunks(text, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'):

    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower()
                  for key, group in itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O') if key]

    return [cand for cand in candidates
            if cand not in stop_words and not all(char in punct for char in cand)]

In [546]:
grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'
chunker = nltk.chunk.regexp.RegexpParser(grammar)

In [548]:
tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(tritext))

In [552]:
len(tagged_sents)

6

In [558]:
for tagged_sent in tagged_sents:
    print len(tagged_sent)
    print tagged_sent[0] # a tuple

26
('Biodegradable', 'JJ')
28
('Much', 'JJ')
24
('Despite', 'IN')
17
('Thorough', 'NNP')
29
('A', 'DT')
36
('In', 'IN')


In [560]:
nltk.chunk.tree2conlltags(chunker.parse(tagged_sents[0]))

[('Biodegradable', 'JJ', u'O'),
 ('Thermogelling', 'VBG', u'O'),
 ('Polymers', 'NNS', u'B-KT'),
 ('for', 'IN', u'I-KT'),
 ('Biomedical', 'NNP', u'I-KT'),
 ('Applications', 'NNP', u'I-KT'),
 ('Thermogelling', 'NNP', u'I-KT'),
 ('polymers', 'NNS', u'I-KT'),
 ('belong', 'VBP', u'O'),
 ('to', 'TO', u'O'),
 ('a', 'DT', u'O'),
 ('class', 'NN', u'B-KT'),
 ('of', 'IN', u'I-KT'),
 ('stimuli-responsive', 'JJ', u'I-KT'),
 ('hydrogels', 'NNS', u'I-KT'),
 ('that', 'WDT', u'O'),
 ('undergo', 'VBP', u'O'),
 ('a', 'DT', u'O'),
 ('macroscopic', 'NN', u'B-KT'),
 ('sol-to-gel', 'JJ', u'B-KT'),
 ('transition', 'NN', u'I-KT'),
 ('in', 'IN', u'I-KT'),
 ('response', 'NN', u'I-KT'),
 ('to', 'TO', u'O'),
 ('temperature', 'NN', u'B-KT'),
 ('.', '.', u'O')]

In [564]:
all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))

In [565]:
len(all_chunks)

160

In [ ]:
candidates = [' '.join(word for word, pos, chunk in group).lower()
              for key, group in itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O') if key]

[cand for cand in candidates
        if cand not in stop_words and not all(char in punct for char in cand)]

In [567]:
a = itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O')

In [ ]:
for key, group in a:
    print key
    print list(group)
# This set of code works the first time.
# Then it no longer prints anything. The itertools.groupby object has been destroyed!!

In [ ]:
# Need to save it to a list first:
a = list((key, list(group)) for key, group in itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O'))
print len(list((key, list(group)) for key, group in itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O'))) # 59
for i in a:
    print len(i[1]) # varying lengths of groups, which are running elements in all_chunks that have the same tag (either 'O' or not)

In [ ]:
# Play around with the lambda: just have 1 input. It doesn't work
a1 = list((key, list(group)) for key, group in itertools.groupby(all_chunks, lambda (chunk): chunk != 'O'))
len(a1[0][1]) # 160. Basically all elements of all_chunks are not 'O'. So it's 1 True for all elements.

In [ ]:
# Unpacking further
for key, group in a:
    if key:
        print group
        print ' '.join(word for word, pos, tag in group)
        print ' '

In [578]:
candidates

['polymers for biomedical applications thermogelling polymers',
 'class of stimuli-responsive hydrogels',
 'macroscopic sol-to-gel transition in response',
 'temperature',
 'ongoing research',
 'fi eld',
 'hydrogels for biomedical applications',
 'injectable sustained drug-release matrix',
 'scaffolds for tissue regeneration',
 'robust developments',
 'polymers in recent decades',
 'fi eld',
 'challenges',
 'optimization of materials properties',
 'thorough investigation',
 'effectiveness of drug delivery',
 'hydrogel-forming polymer carriers',
 'highlighted case study on oncogel',
 'experimental drug delivery depot formulation',
 'light',
 'shortcomings',
 'polymers as drug delivery systems',
 'article',
 'developments in biodegradable thermoresponsive polymers',
 'biomedical applications',
 'years',
 'focus on materials/ technical challenges',
 'approaches',
 'problems']

In [ ]:
# Get candidate keywords
extract_candidate_chunks(tritext)
set(extract_candidate_chunks(tritext))
# The same as extract_candidate_chunks(tritext), except arranged alphabetically
'''
boc_texts = [extract_candidate_chunks(text) for text in tritexts]
dictionary = gensim.corpora.Dictionary(boc_texts)
cp = [dictionary.doc2bow(boc_text) for boc_text in boc_texts]'''

# Doesn't get me a list of keyphrases...

## Using words (and scikit-learn tfidf)

A different approach using a combination of NLTK and scikit-learn. As well as using words not chunks.

### 1. Try using 1st three documents as corpus:

In [1]:
tritexts = [PRF_IMRE2.Title_of_Paper.iloc[0] + PRF_IMRE2.Significance_of_Paper.iloc[0],
            PRF_IMRE2.Title_of_Paper.iloc[1] + PRF_IMRE2.Significance_of_Paper.iloc[1],
            PRF_IMRE2.Title_of_Paper.iloc[2] + PRF_IMRE2.Significance_of_Paper.iloc[2]]

tritext = PRF_IMRE2.Title_of_Paper.iloc[1] + PRF_IMRE2.Significance_of_Paper.iloc[1]

NameError: name 'PRF_IMRE2' is not defined

In [ ]:
'''def get_tokens(text):
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    no_punctuation = lowers.translate(None, string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

tokens = get_tokens(tritext)
print tokens
# count = Counter(tokens)
# print count.most_common(10)

token_noStop = [w for w in tokens if not w in stopwords.words('english')]
print token_noStop
# count = Counter(filtered)
# print count.most_common(100)

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
stemmed = stem_tokens(token_noStop, stemmer)
print stemmed
# count = Counter(stemmed)
# print count.most_common(100)
'''

In [ ]:
'''# try snowball stemming
a = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(tritext)]
for i in a:
    print [SnowballStemmer("english").stem(word) for word in i]
    print ' ''''

In [ ]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text = tritext): # Need to change this argument for each extraction
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

stemmer = PorterStemmer()
token_dict = {}
for ind, doc in enumerate(tritexts):
    lowers = doc.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[ind] = no_punctuation # tokenized corpus (but not stemmed)

In [ ]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(tritexts)
tfs2 = tfidf.fit_transform(token_dict.values())
# Note that token_dict.values() is a preprocessed corpus (lowercased and punctuation-removed)
# tritexts is not.

In [ ]:
# 2 ways to visualize the sparse matrix
print tfs
tfs.todense()

In [ ]:
# Get idf only
idf = tfidf.idf_
print dict(zip(tfidf.get_feature_names(), idf))

In [ ]:
for i in tfs:
    print i.shape
# 95 keywords, or features, were created from the corpus.
# Each document (there are 3) has a distribution of some of these words. Most are zeroes.
# Therefore tfs2 is a sparse matrix.

In [ ]:
m = tfs.todense()
m[0, :13]
# Out of the first 13 keywords, Document 1 contains the 12th and none of the rest.

In [ ]:
# The first document
a1 = tfs[0, :].toarray()

# These are the indices and keywords belonging to doc 1 (11 out of 95)
for ind, i in enumerate(a1[0]):
    if i != 0:
        print ind
        print i
        print tfidf.get_feature_names()[ind]


In [ ]:
doc1_tfidf = dict(zip(tfidf.get_feature_names(), a1.tolist()[0]))
doc1_tfidf

In [ ]:
doc1_tfidf_dense = pd.DataFrame(
    {'feature_names': [tfidf.get_feature_names()[ind] for ind, i in enumerate(m[0, :].tolist()[0]) if i != 0],
     'tfidf': [i for ind, i in enumerate(m[0, :].tolist()[0]) if i != 0]
    })

doc1_tfidf_dense.sort('tfidf', ascending = False)

In [ ]:
doc2_tfidf_dense = pd.DataFrame(
    {'feature_names': [tfidf.get_feature_names()[ind] for ind, i in enumerate(m[1, :].tolist()[0]) if i != 0],
     'tfidf': [i for ind, i in enumerate(m[1, :].tolist()[0]) if i != 0]
    })

doc2_tfidf_dense.sort('tfidf', ascending = False)

In [ ]:
doc3_tfidf_dense = pd.DataFrame(
    {'feature_names': [tfidf.get_feature_names()[ind] for ind, i in enumerate(m[2, :].tolist()[0]) if i != 0],
     'tfidf': [i for ind, i in enumerate(m[2, :].tolist()[0]) if i != 0]
    })

doc3_tfidf_dense.sort('tfidf', ascending = False)

In [ ]:
print doc1_tfidf_dense.shape
print doc2_tfidf_dense.shape
print doc3_tfidf_dense.shape

### 2. Try fitting another document among the 3-doc-corpus:

In [ ]:
tritext1 = PRF_IMRE2.Title_of_Paper.iloc[2] + PRF_IMRE2.Significance_of_Paper.iloc[2]

In [ ]:
def tokenize(text = tritext1):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

stemmer = PorterStemmer()
token_dict = {}
for ind, doc in enumerate(tritexts):
    lowers = doc.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[ind] = no_punctuation # tokenized corpus

In [ ]:
tfidf1 = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs1 = tfidf1.fit_transform(token_dict.values())

In [ ]:
for i in tfs1:
    print i.shape

In [ ]:
# The first document
a1 = tfs1[0, :].toarray()

# These are the indices and keywords belonging to doc 1 (11 out of 95)
for ind, i in enumerate(a1[0]):
    if i != 0:
        print ind
        print i
        print tfidf1.get_feature_names()[ind]

Changing the tokenize(text), ie the text to be fitted, seems to not affect the tfidf fitting result. The fitting seems to just conjure a bunch of keywords out of the corpus, and assign the keywords to each of the docs. You figure out which is the doc you had been interested in in the first place.

### 3. Now try increasing the corpus size:

In [ ]:
tritexts1 = [PRF_IMRE2.Title_of_Paper.iloc[0] + PRF_IMRE2.Significance_of_Paper.iloc[0],
             PRF_IMRE2.Title_of_Paper.iloc[1] + PRF_IMRE2.Significance_of_Paper.iloc[1],
             PRF_IMRE2.Title_of_Paper.iloc[2] + PRF_IMRE2.Significance_of_Paper.iloc[2],
             PRF_IMRE2.Title_of_Paper.iloc[3] + PRF_IMRE2.Significance_of_Paper.iloc[3]]

tritext1 = PRF_IMRE2.Title_of_Paper.iloc[1] + PRF_IMRE2.Significance_of_Paper.iloc[1]

In [2]:
def tokenize(text = tritext1):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

stemmer = PorterStemmer()
token_dict = {}
for ind, doc in enumerate(tritexts1):
    lowers = doc.lower()
    no_punctuation = lowers.translate(None, string.punctuation)
    token_dict[ind] = no_punctuation # tokenized corpus

NameError: name 'tritext1' is not defined

In [ ]:
tfidf2 = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs2 = tfidf2.fit_transform(token_dict.values())

In [ ]:
for i in tfs2:
    print i.shape
# The vocabulary increases to 124.

In [ ]:
# The first document
a1 = tfs2[0, :].toarray()

# These are the indices and keywords belonging to doc 1 (11 out of 95)
for ind, i in enumerate(a1[0]):
    if i != 0:
        print ind
        print i
        print tfidf2.get_feature_names()[ind]

#### Trouble shoot document 5. It seems to have encoding problems.

In [ ]:
s = PRF_IMRE2.Title_of_Paper.iloc[4] + PRF_IMRE2.Significance_of_Paper.iloc[4]
print s
# Part of the string is not ascii: '\xe8\xcf'

In [ ]:
s.decode('ascii') # returns error

In [ ]:
s[399]

In [ ]:
# Remove non ascii characters
printable = set(string.printable)
s = filter(lambda x: x in printable, s)
print s

In [ ]:
# Check that, for a string with ALL normal ascii characters, the filtering doesn't cause harm
s1 = PRF_IMRE2.Title_of_Paper.iloc[3] + PRF_IMRE2.Significance_of_Paper.iloc[3]
filter(lambda x: x in printable, s1) == s1

In [ ]:
# Find out which abstracts have non-ascii characters in them.
for ind, i in enumerate(PRF_IMRE2.Significance_of_Paper):
    if filter(lambda x: x in printable, i) != i:
        print ind
# All of these papers have non-ascii characters in them.

In [ ]:
corpus = []
printable = set(string.printable)

for i, j in zip(PRF_IMRE2.Title_of_Paper, PRF_IMRE2.Significance_of_Paper):
    encoding_fixed = filter(lambda x: x in printable, i+j)
    corpus.append(encoding_fixed)


In [ ]:
for ind, i in enumerate(corpus):
    if filter(lambda x: x in printable, i) != i:
        print ind
# None.